### NICO

Implementation based on the matlab code by M. Rabbat

In [1]:
import numpy as np
from random import shuffle, random
from scipy.special import comb
from scipy.sparse import csr_matrix
import itertools

Initialize parameters

In [2]:
n = 3 #number of nodes in the network
T = 100 #number of paths
Nm = 5 #number of nodes per path
# np.random.seed(2)
np.random.seed(1337)

In [3]:
A = np.random.rand(n, n)
A = A / A.sum(axis = 1, keepdims=True)

In [4]:
pi = np.random.rand(n, 1)
pi = pi / pi.sum(axis=0, keepdims=True)

Generate some paths according to this Markov model

In [5]:
X = np.zeros((T, Nm))

In [6]:
# Generate random numbers for testing purposes. Change to random() when it is ready
R_out = np.random.rand(T,1)
R_in = np.random.rand(T, Nm)

In [7]:
cumprobs = pi.cumsum(axis = 0)

In [8]:
iterator = 0
for walk in X:
#     Sample the starting node from Pi
#     larger = (cumprobs >= random()).nonzero()
    larger = (cumprobs >= R_out[iterator][0]).nonzero()
    walk[0] = larger[0][0]
#     Sample remaining nodes in the path by taking a random walk
    for i in range(1, Nm):
        cumprobs_in = A[int(walk[i - 1]),:].cumsum(axis=0)
#       larger = (cumprobs >= random()).nonzero()
        larger = (cumprobs_in >= R_in[iterator][i]).nonzero()
        walk[i] = larger[0][0]
    iterator += 1
X = X.astype(int)

Shuffle observations

In [9]:
# Y = X.copy()
# for walk in Y:
#     shuffle(walk)

In [10]:
# numTrials = 50

Utils

In [11]:
def normalize_csr_rows(csr_mat):
    row_sums = np.array(csr_mat.sum(axis=1))[:,0]
    row_indices, col_indices = csr_mat.nonzero()
    csr_mat.data /= row_sums[col_indices]

In [12]:
def compute_likelihood(bag_of_nodes, pi_hat, A_hat):
    n = len(bag_of_nodes)
    
    permutation_orders = list(itertools.permutations(list(range(n))))
    
    gamma = np.zeros(n)
    Gamma = np.zeros((n,n))
    
    for order in permutation_orders:
        starting_node = order[0]
        p = pi_hat[bag_of_nodes[starting_node]]
        
        for i in range(1, n):
            p = p * A_hat[bag_of_nodes[order[i - 1]], bag_of_nodes[order[i]]]
        
        gamma[order[0]] += p
        
        for i in range(1, n):
            Gamma[order[i - 1]][order[i]] += p
    return gamma, Gamma

NICO implementation

In [13]:
# @TBD Change names. Come up with something meaningful and readable

def nico(X, n):
#     T = np.shape(X)[0]
    
    #number of nodes in each path
#     size = lambda array: len(array)
#     Nm = np.apply_along_axis(size, 1, X)
    
    #Init pi_hat
    #Assume all states appear at least once in the data
    pi_hat = 1 + 0.3 * np.random.rand(n, 1)
    pi_hat = pi_hat / pi_hat.sum(axis = 0, keepdims = True)
    pi_hat = [item[0] for item in pi_hat]
    
    # Construct A_hat as a sparse matrix
    # First determine an upperbound on the number of non-zero entries
    ii = []
    jj = []

    for walk in X:
        V = np.array(list(itertools.combinations(walk, 2)))
        ii.append(list(V[:, 0]))
        jj.append(list(V[:, 1]))

    ii = [item for sublist in ii for item in sublist]
    jj = [item for sublist in jj for item in sublist]
    ss = np.ones(len(ii))
    
    A_hat = csr_matrix((ss, (ii, jj)), shape = (n,n))
    A_hat = (A_hat + A_hat.transpose()) / 2
    A_hat_copy = A_hat.copy()
    A_hat_copy.data.fill(1)
    
    A_hat = A_hat_copy + 0.4 * csr_matrix((np.random.random((A_hat.nnz)),A_hat.nonzero()), shape=A_hat.shape)
    A_hat = A_hat.transpose()
    
    #Normalize A_hat
    normalize_csr_rows(A_hat)
    
    # E-STEP
    #Test on one permutation
    r_alpha_gamma = []
    r_alpha_Gamma = []
#     gamma, Gamma = compute_likelihood(X[0], pi_hat, A_hat)
    
    for bag_of_nodes in X:
        gamma, Gamma = compute_likelihood(bag_of_nodes, pi_hat, A_hat)
        gamma_sum = sum(gamma)
        r_alpha_gamma.append(gamma/gamma_sum)
        r_alpha_Gamma.append(Gamma/gamma_sum)
        
    return r_alpha_gamma, r_alpha_Gamma

In [14]:
g, G = nico(X, n)

In [16]:
g[0]

array([0.19951574, 0.20032284, 0.20032284, 0.19951574, 0.20032284])

In [17]:
G[-2]

array([[0.        , 0.19079132, 0.19079132, 0.21675629, 0.19079132],
       [0.19457599, 0.        , 0.20904726, 0.19457599, 0.20904726],
       [0.19457599, 0.20904726, 0.        , 0.19457599, 0.20904726],
       [0.21675629, 0.19079132, 0.19079132, 0.        , 0.19079132],
       [0.19457599, 0.20904726, 0.20904726, 0.19457599, 0.        ]])